In [1]:
import requests
import json
import os

from dotenv import load_dotenv

from datetime import datetime

from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [3]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

## Prompt
- load prompt
- load template with prompt

In [17]:
with open("prompt_continuouseffects.txt", "r") as file:
    prompt_continuous_effects = file.read()
print(prompt_continuous_effects)

prompt_template = PromptTemplate.from_template(prompt_continuous_effects)
#prompt_template = ChatPromptTemplate.from_template(prompt_continuous_effects)

"""
You will get two cards from the cardgame Magic: the Gathering. {card1} and {card2} both have continuous effects.
You can assume that {card1} is played first, therefor has the earlier timestamp.
Use the {rules_db} as context for your decision.
You will get a {question} about two cards and have to make a decision.

You will follow these steps:

1. Give the oracle texts from both cards.
2. Determine the layer on which the effects of the cards are applied. Layers are defined in 613.1 in the {rules_db}.
3. Decide which you rule fitting: Timestamp and Layers (613.1 to 613.7), Dependency (613.8) or Overriding of effects (613.9).
4. Give your judging: The continuous effect of which card is applied in game: {card1} or {card2}? Quote the effects.
5. Explain why you decided this way.
6. Quote the rule you used for your decision.

"""


## RAG
= Retrieved Augmented Generation
- ChatGPT's knowlegde of the rules is from January 2022: for current state the rulebook needs to be retrieved and passed to the LLM
- this data can provide context for the specific questions
### Loader
- simple loader to load .txt data: complete rule book from Magic: The Gathering

In [5]:
#loader = TextLoader("./data/rules_shortened.txt")
loader = TextLoader("./data/rules.txt")
rules_doc = loader.load()

### Splitter
- splitting the text into smaller chunks
- this text splitter splits text recursively by characters: ["\n\n", "\n", " ", ""], so first paragraphs, then sentences, then words
- keeps semantically related pieces together as long as possible

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
rules_splits = text_splitter.split_documents(rules_doc)
# print(rules_splits[0])

### Vector Store
- for storage of the embeddings
- embedding for capture semantic meaning of the text: that's how context is given for later question
- OpenAI embedding for ChatGPT
- Chroma as open-source embedding database
- finally, construct a retriever out of vectorstore 

In [7]:
rules_db = Chroma.from_documents(documents=rules_splits, embedding=OpenAIEmbeddings())
retriever_rules = rules_db.as_retriever()

- create new LLM: temperature 0 for more consistent, non-creative and therefor comparable output

In [8]:
# llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
model = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

## Case 1: Blood Moon vs. Urborg, Tomb of Yawgmoth
- Get card information via scryfall.com and delete unnecessary data: 
- keep only: name, mana cost, converted mana cost, type, oracle text, colors, keywords

In [38]:
card1 = "Urborg, Tomb of Yawgmoth"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "rulings_uri", "set_search_uri", "scryfall_set_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
        
    # convert into json dict
    card1 = json.loads(json.dumps(data))
    
    # check for additional rulings via id
    card_id = card1.get('id')
    if card_id:
        response = requests.get(f'https://api.scryfall.com/cards/{card_id}/rulings', 
                 headers={'Accept': 'application/json'})
    
    # add additional rules to card
        if response.status_code == 200:
            data_rulings = response.json()
            comments = [item.get('comment') for item in data_rulings.get('data', [])]
            card1['comments'] = comments
        
    with open('./data/cards/urborg.json', 'w') as f:
        json.dump(card1, f)
    print(card1)


{'id': '9e1a9e38-6ffc-490f-b0be-23ba4e8204c6', 'name': 'Urborg, Tomb of Yawgmoth', 'mana_cost': '', 'cmc': 0.0, 'type_line': 'Legendary Land', 'oracle_text': 'Each land is a Swamp in addition to its other land types.', 'colors': [], 'keywords': [], 'produced_mana': ['B'], 'comments': ["Urborg, Tomb of Yawgmoth isn't a Swamp while it's not on the battlefield.", "Land cards not on the battlefield aren't Swamps while Urborg is on the battlefield.", 'Urborg\'s ability causes each land on the battlefield to have the land type Swamp. Any land that\'s a Swamp has the ability "{T}: Add {B}." Nothing else changes about those lands, including their names, other subtypes, other abilities, and whether they\'re legendary, basic, or snow.', "If an effect such as that of Magus of the Moon causes Urborg to lose its abilities by setting it to a basic land type not in addition to its other types, it won't turn lands into Swamps, no matter in what order those effects started to apply."]}


In [39]:
card2 = "Blood Moon"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict
    card2 = json.loads(clean_json)
    
    # check for additional rulings via id
    card_id = card2.get('id')
    if card_id:
        response = requests.get(f'https://api.scryfall.com/cards/{card_id}/rulings', 
                 headers={'Accept': 'application/json'})
    
    # add additional rules to card
        if response.status_code == 200:
            data_rulings = response.json()
            comments = [item.get('comment') for item in data_rulings.get('data', [])]
            card2['comments'] = comments
    
    with open('./data/cards/bloodmoon.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

{'id': 'd072e9ca-aae7-45dc-8025-3ce590bae63f', 'name': 'Blood Moon', 'mana_cost': '{2}{R}', 'cmc': 3.0, 'type_line': 'Enchantment', 'oracle_text': 'Nonbasic lands are Mountains.', 'colors': ['R'], 'keywords': [], 'comments': ['If a nonbasic land has an ability that triggers “when” it enters the battlefield, it will lose that ability before it can trigger.', 'If a nonbasic land has an ability that causes it to enter the battlefield tapped, it will lose that ability before it can apply. The same is also true of any other abilities that modify how a land enters the battlefield or apply “as” a land enters the battlefield, such as the first ability of Cavern of Souls.', 'Nonbasic lands will lose any other land types and abilities they had. They will gain the land type Mountain and gain the ability “{T}: Add {R}.”', "This effect doesn't affect names or supertypes. It won't turn any land into a basic land or remove the legendary supertype from a legendary land, and the lands won't be named “M

In [ ]:
# load card variables from saved data
with open("./data/cards/urborg.txt", "r") as file:
    card1 = file.read()

with open("./data/cards/bloodmoon.txt", "r") as file:
    card2 = file.read()

In [22]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

- Use Chatmodel instead of LLM: more possible token in prompt
- Outputparser to parse message response from Chatmodel to str
- check testcases.md to compare expected outcome with ChatGPT's answer

In [23]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the final board state of two cards played with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts:
- Urborg, Tomb of Yawgmoth: "Each land is a Swamp in addition to its other land types."
- Blood Moon: "Nonbasic lands are Mountains."

2. Layer determination:
The effects of these cards fall under the category of type-changing effects, which are applied in layer 4 according to rule 613.1.

3. Decision on ruling:
In this scenario, we will apply the rule of Dependency (613.8) as the effects of both cards are dependent on each other. The outcome of one effect (making lands Swamps) affects the outcome of the other effect (making nonbasic lands Mountains).

4. Judging:
The continuous effect of the card that is applied in the game is Blood Moon: "Nonbasic lands are Mountains."

5. Explanation:
Since the effects of Urborg, Tomb of Yawgmoth and Blood Moon are dependent on each other, we need to consider the dependency rule (613.8). In this case, the effect of Blood Moon overrides the effect of Urborg, Tomb of Yawgmoth because Blood Moon specifically states that nonbasic lands 

- saving the answers as .txt for comparison by date & time: 

In [24]:
current_datetime = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
file_name = current_datetime+".txt"

with open(f'./data/answers/case1_{file_name}', 'w') as f:
    f.write(answer)

In [22]:
# cleanup
rules_db.delete_collection()

## Case 2a: Opalescence vs. Humility
- timestamp rule determines the outcome: *Humility* is played last and it's effect overrides *Opalescence*

In [42]:
card1 = "Opalescence"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict 
    card1 = json.loads(clean_json)
    
    # check for additional rulings via id
    card_id = card1.get('id')
    if card_id:
        response = requests.get(f'https://api.scryfall.com/cards/{card_id}/rulings', 
                 headers={'Accept': 'application/json'})
    
    # add additional rules to card
        if response.status_code == 200:
            data_rulings = response.json()
            comments = [item.get('comment') for item in data_rulings.get('data', [])]
            card1['comments'] = comments
    
    with open('./data/cards/opalescence.json', 'w') as f:
        json.dump(card1, f)
    print(card1)

{'id': '3c0071fb-afa5-47b5-b266-2b10a4f5a98a', 'name': 'Opalescence', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.', 'colors': ['W'], 'keywords': [], 'comments': ['This is the current interaction between Humility and Opalescence: The type-changing effect applies at layer 4, but the rest happens in the applicable layers. The rest of it will apply even if the permanent loses its ability before it’s finished applying. So if Opalescence, Humility, and Worship are on the battlefield and Opalescence entered the battlefield before Humility, the following is true: Layer 4: Humility and Worship each become creatures that are still enchantments. (Opalescence). Layer 6: Humility and Worship each lose their abilities. (Humility) Layer 7b: Humility becomes 4/4 and Worship becomes 4/4. (Opalescence). Humility becomes 1/1

In [41]:
card2 = "Humility"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    print(card2)
    keywords = ["object", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict 
    card2 = json.loads(clean_json)
    
    # check for additional rulings via id
    card_id = card2.get('id')
    if card_id:
        response = requests.get(f'https://api.scryfall.com/cards/{card_id}/rulings', 
                 headers={'Accept': 'application/json'})
    
    # add additional rules to card
        if response.status_code == 200:
            data_rulings = response.json()
            comments = [item.get('comment') for item in data_rulings.get('data', [])]
            card2['comments'] = comments
    
    with open('./data/cards/humility.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

Humility
{'id': '55ad6a45-a840-45ba-89ad-066e20e983f3', 'name': 'Humility', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'All creatures lose all abilities and have base power and toughness 1/1.', 'colors': ['W'], 'keywords': [], 'comments': ['This is the current interaction between Humility and Opalescence: The type-changing effect applies at layer 4, but the rest happens in the applicable layers. The rest of it will apply even if the permanent loses its ability before it’s finished applying. So if Opalescence, Humility, and Worship are on the battlefield and Opalescence entered the battlefield before Humility, the following is true: Layer 4: Humility and Worship each become creatures that are still enchantments. (Opalescence). Layer 6: Humility and Worship each lose their abilities. (Humility) Layer 7b: Humility becomes 4/4 and Worship becomes 4/4. (Opalescence). Humility becomes 1/1 and Worship becomes 1/1 (Humility). But if Humility entered the ba

In [ ]:
with open("./data/cards/opalescence.txt", "r") as file:
    card1 = file.read()

with open("./data/cards/humility.txt", "r") as file:
    card2 = file.read()

In [29]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [30]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts:
- Opalescence: Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.
- Humility: All creatures lose all abilities and have base power and toughness 1/1.

2. The effects of these cards are applied in different layers according to the rules. The effect of Opalescence is applied in layer 4, which deals with type-changing effects. On the other hand, the effect of Humility is applied in layer 6, which deals with ability-removing effects.

3. I will rule based on the concept of Layers (613.1 to 613.7) as it determines the order in which continuous effects are applied based on their respective layers.

4. Judging: The continuous effect of Opalescence is applied in the game. The effect of Opalescence turns all other non-Aura enchantments into creatures with power and toughness equal to their mana value.

5. Explanation: Since Opalescence's effect is applied in layer 4, which is earlier t

In [111]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
# file_name = current_datetime+".txt"

with open(f'./data/answers/case2_{file_name}.txt', 'w') as f:
    f.write(answer)

## Case 2b: Humility vs. Opalescence 
- because timestamp is important here, testing the case with changed order

In [ ]:
with open("./data/cards/humility.txt", "r") as file:
    card1 = file.read()

with open("./data/cards/opalescence.txt", "r") as file:
    card2 = file.read()

In [35]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [36]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts:
- Humility: All creatures lose all abilities and have base power and toughness 1/1.
- Opalescence: Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.

2. The effects of these cards are applied in different layers according to the layer system in Magic: the Gathering. The effect of Humility, which removes all creature abilities and sets their power and toughness to 1/1, is applied in Layer 6 (ability-removing effects) and Layer 7b (power/toughness setting effects). On the other hand, the effect of Opalescence, which turns non-Aura enchantments into creatures with power and toughness equal to their mana value, is applied in Layer 4 (type-changing effects) and Layer 7b (power/toughness setting effects).

3. I will use the rule of Overriding of effects (613.9) to determine which continuous effect takes precedence in this scenario.

4. Judging: The continuous effect of Humility, wh

In [37]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
# file_name = current_datetime+".txt"

with open(f'./data/answers/case2_{file_name}.txt', 'w') as f:
    f.write(answer)

In [144]:
# cleanup
rules_db.delete_collection()